# Capex Project Announcement Trends

## State-Wise Trends



In [2]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
from plotly import tools

import os 

# Folder path containing the XLSX files
folder_path = '/Users/kalyan/Library/CloudStorage/OneDrive-Personal/Kalyan/KK-Python/Kalyan-Jupyter-Notebooks/Capex/raw_data/'

filename_1 = "Agg_Ann_no. of projects copy.xlsx" 
file_path_1 = os.path.join(folder_path, filename_1)

df = pd.read_excel(file_path_1 , sheet_name='Sheet1', usecols= "A:R")

print(df.columns)


# Group DataFrame by Category
grouped_df = df.groupby('Region')

# print(grouped_df)

# Create subplots with shared x-axis and y-axis
fig = make_subplots(rows=len(grouped_df), cols=1, shared_xaxes=False, shared_yaxes=True)

# Iterate over each category
for i, (category, category_data) in enumerate(grouped_df):
#     print(i,category, category_data)
    # Add scatter trace for each state in the category
    for _, row in category_data.iterrows():
        fig.add_trace(go.Scatter(x=df.columns[2:], y=row.values[2:], name=row['State'], legendgroup = i), row=i + 1, col=1)
        # Update subplot title
        fig.update_yaxes(title_text="Region: " + category, row=i + 1, col=1)
#         fig.update_xaxes(title_text="Quarters", row=i, col=1)

# Update layout
fig.update_layout(height=1000, width=1000, title_text='Capex announcements by Indian states Region', 
                  showlegend=True, legend_tracegroupgap = 75,)

# Show the plot
fig.show()


Index(['State', 'june-2019', 'sep-2019', 'dec-2019', 'march-2020', 'june-2020',
       'sep-2020', 'dec-2020', 'march-2021', 'june-2021', 'sep-2021',
       'dec-2021', 'march-2022', 'june-2022', 'sep-2022', 'dec-2022',
       'march-2023', 'year-2019'],
      dtype='object')


KeyError: 'Region'

In [109]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the DataFrame with year-on-year growth rates
# df_growth = pd.read_csv('your_data.csv')  # Replace 'your_data.csv' with the actual file path or DataFrame

# Sample DataFrame
data = {
    'State': ['AP', 'Bihar', 'Kashmir', 'Delhi'], 
    'Category': ['A', 'B', 'C', 'C'],
    'june-2019': [23,34,51,45],
    'sep-2019': [34,13,45,23],
    'dec-2019': [67, 25, 86, 16],
    'mar-2020': [76,34,61,15],
    'june-2020': [76,44,11,5],
    'sep-2020': [84,53,45,3],
    'dec-2020': [97, 65, 66, 6],
    'mar-2021':[99, 75, 56, 6],
    'june-2021': [106,64,12,6],
    'sep-2021': [104,73,15,8],
    'dec-2021': [127, 85, 86, 12],
    'mar-2022': [137, 55, 26, 22],
}

df = pd.DataFrame(data)
# Define the features (X) and the target variable (y)
features = df.drop(['State', 'Category'], axis=1)  # Exclude non-feature columns
df['type']=""
target = df['type']  # Assuming 'type' is the target variable column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create a Random Forest Classifier model
model = RandomForestClassifier()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(df)

<bound method BaseForest.fit of RandomForestClassifier()>
Accuracy: 1.0
     State Category  june-2019  sep-2019  dec-2019  mar-2020  june-2020  \
0       AP        A         23        34        67        76         76   
1    Bihar        B         34        13        25        34         44   
2  Kashmir        C         51        45        86        61         11   
3    Delhi        C         45        23        16        15          5   

   sep-2020  dec-2020  mar-2021  june-2021  sep-2021  dec-2021  mar-2022 type  
0        84        97        99        106       104       127       137       
1        53        65        75         64        73        85        55       
2        45        66        56         12        15        86        26       
3         3         6         6          6         8        12        22       


### Announced Project cost
Question: Is there a (seasonal / quarterly) trend in project announcements ?

In [25]:

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
from plotly import tools
import plotly.express as px
import os 

# Folder path containing the XLSX files
folder_path = '/Users/kalyan/Library/CloudStorage/OneDrive-Personal/Kalyan/KK-Python/Kalyan-Jupyter-Notebooks/Capex/proc_data'

# filename_1 = "Project_CapEx_Final.xlsx"
filename_1 = "Final_Capex.xlsx"

file_path_1 = os.path.join(folder_path, filename_1)

df = pd.read_excel(file_path_1 , sheet_name='Sheet1')

# Split the FY column to extract the fiscal year
df[['FY_start', 'FY_end']] = df['FY'].str.split('-', expand=True)

# Add a fiscal year column
# df['Fiscal_Year'] = 'FY'+ df['FY_start'] + '-Q' + df['Quarter'].astype(str)
df['Fiscal_Year'] = 'FY'+ df['FY_start'] 

# Create a color map for quarters
quarter_colors = {1: 'blue', 2: 'green', 3: 'red', 4: 'purple'}

# Map colors to quarters
df['Color'] = df['Quarter'].map(quarter_colors)

print(df.columns)

# Group by Fiscal Year and Quarter, calculate the sum of values
grouped_df = df.groupby(['Fiscal_Year', 'Quarter'])['Cost (Rs.million)'].sum().reset_index()

print(grouped_df.columns)
# Create the bar chart using Plotly
fig = px.bar(grouped_df, x='Fiscal_Year', y='Cost (Rs.million)', color='Quarter', barmode='group',
             labels={'Fiscal_Year': 'Fiscal Year', 'Cost (Rs.million)': 'Announcement Value'},
             title='Cost of announced project (by Quarter)',
             color_discrete_sequence=px.colors.qualitative.Set1,
            )

# Set x-axis labels to include both FY and Quarter
fig.update_xaxes(tickmode='array', 
                 tickvals=grouped_df.index,
#                  ticktext=[f"{row['Fiscal_Year']} Q{row['Quarter']}" for idx, row in grouped_df.iterrows()]
                )

# Show the plot
fig.show()


Index(['Company Name', 'Project Name', 'Cost (Rs.million)', 'Project Status',
       'Last Updated On', 'Industry Group', 'Ownership Group', 'Project Type',
       'Company Code', 'Project No.', 'FY', 'Quarter', 'Type', 'Location',
       'District', 'State', 'Products', 'Capacity', 'Units',
       'Reason for Stalling of Projects', 'Sector', 'Owner Affliation',
       'Foreign', 'st_ut_code', 'region', 'IndG_L0', 'IndG_L4', 'IndG_L8',
       'Gross Fiscal Deficit', 'Revenue Deficit', 'Primary Deficit',
       'Own Tax Revenue', 'Own Non-Tax Revenue', ' Interest Payments',
       'State-wise Pension', 'State-wise Capital Expenditure',
       'Capital Outlay', 'Social Sector Expenditure',
       ' Gross State Domestic Product', 'GSVA-Agriculture',
       'GSVA-Manufacturing', 'GSVA-Construction', 'GSVA-Industry',
       'GSVA-Banking and Insurance ', 'GSVA-Services ',
       'Per Capita Net State Domestic Product', 'Net State Domestic Product',
       'NSVA-Agriculture', 'NSVA-Manufactu

### Question: What is current status of projects announced in each FY?

In [15]:
grouped_df = df.groupby(['Fiscal_Year','Type'])['Project Name'].count().reset_index()

# Create the bar chart using Plotly
fig = px.bar(grouped_df, x='Fiscal_Year', y='Project Name', color='Type',barmode='group',
#              color_discrete_map=quarter_colors,
             labels={'Fiscal_Year': 'Fiscal Year', 'Project Name': 'Project Announcements(count)'},
             title='Status of Project Announcements',
#              color_discrete_sequence=px.colors.qualitative.Set1,
            )

# Set x-axis labels to include both FY and Quarter
fig.update_xaxes(tickmode='array', 
                 tickvals=grouped_df.index,
#                  ticktext=[f"{row['Fiscal_Year']} Q{row['Quarter']}" for idx, row in grouped_df.iterrows()]
                )

# Show the plot
fig.show()

### Question: Who has been investing in Capex Projects i.e. project owner affliation (Pvt vs Govt)
Ans: In terms of number of announcements, Government led the pack in FY 2014 and 2015 and the next part of the decade, the private sector announcements exceeded the government the rest of the time until end of FY 2022.

Ans: Part-B: In terms of committed spending, Government announcements far exceeded the provate sector until FY 2020,  the private sector announcements matched the government's spending commitments in FY 2021 and exceeded the government only in FY 2022.


In [16]:
# grouped_df = df.groupby(['Fiscal_Year','Owner Affliation'])['Project Name'].count().reset_index()
grouped_df = df.groupby(['Fiscal_Year','Owner Affliation'])['Cost (Rs.million)'].sum().reset_index()

# # Create the bar chart using Plotly
# fig = px.bar(grouped_df, x='Fiscal_Year', y='Project Name', color='Owner Affliation',barmode='group',
# #              color_discrete_map=quarter_colors,
#              labels={'Fiscal_Year': 'Fiscal Year', 'Project Name': 'Project Announcements(count)'},
#              title='Project Announcements (Count)',
# #              color_discrete_sequence=px.colors.qualitative.Set1,
#             )

fig = px.bar(grouped_df, x='Fiscal_Year', y='Cost (Rs.million)', color='Owner Affliation',barmode='group',
#              color_discrete_map=quarter_colors,
             labels={'Fiscal_Year': 'Fiscal Year', 'Cost (Rs.million)': 'Project Cost'},
             title='Project Announcements (Cost)',
#              color_discrete_sequence=px.colors.qualitative.Set1,
            )

# Set x-axis labels to include both FY and Quarter
fig.update_xaxes(tickmode='array', 
                 tickvals=grouped_df.index,
#                  ticktext=[f"{row['Fiscal_Year']} Q{row['Quarter']}" for idx, row in grouped_df.iterrows()]
                )

# Show the plot
fig.show()

#### Question: What is the proportion of announcements and value of Capex spending commitment by Foreign firms?
Ans: Very minimal, if number of announcements are considered.

In [17]:
# grouped_df = df.groupby(['Fiscal_Year','Foreign'])['Project Name'].count().reset_index()
grouped_df = df.groupby(['Fiscal_Year','Foreign'])['Cost (Rs.million)'].sum().reset_index()

# # Create the bar chart using Plotly
# fig = px.bar(grouped_df, x='Fiscal_Year', y='Project Name', color='Foreign',barmode='group',
# #              color_discrete_map=quarter_colors,
#              labels={'Fiscal_Year': 'Fiscal Year', 'Project Name': 'Project Announcements(count)'},
#              title='Project Announcements (Count)',
# #              color_discrete_sequence=px.colors.qualitative.Set1,
#             )
# Create the bar chart using Plotly
fig = px.bar(grouped_df, x='Fiscal_Year', y='Cost (Rs.million)', color='Foreign',barmode='group',
#              color_discrete_map=quarter_colors,
             labels={'Fiscal_Year': 'Fiscal Year', 'Project Name': 'Project Announcements(count)'},
             title='Project Announcements (Count)',
#              color_discrete_sequence=px.colors.qualitative.Set1,
            )
# Set x-axis labels to include both FY and Quarter
fig.update_xaxes(tickmode='array', 
                 tickvals=grouped_df.index,
#                  ticktext=[f"{row['Fiscal_Year']} Q{row['Quarter']}" for idx, row in grouped_df.iterrows()]
                )

# Show the plot
fig.show()

#### What is the proportion of manufacturing to service project announcements?
Ans: There is a steady decline in announced cost of services projects, but manufacturing declined between 2016 and 2020, and rose to the pre-pandemic level by FY 2022.

In [18]:
grouped_df = df.groupby(['Fiscal_Year','Sector'])['Cost (Rs.million)'].sum().reset_index()

# Create the bar chart using Plotly
fig = px.line(grouped_df, x='Fiscal_Year', y='Cost (Rs.million)', color='Sector',
             labels={'Fiscal_Year': 'Fiscal Year', 'Project Name': 'Project Announcements(count)'},
             title='Project Announcements (Cost)',
             color_discrete_sequence=px.colors.qualitative.Set1,
            )
# Set x-axis labels to include both FY and Quarter
fig.update_xaxes(tickmode='array', 
                 tickvals=grouped_df.index,
#                  ticktext=[f"{row['Fiscal_Year']} Q{row['Quarter']}" for idx, row in grouped_df.iterrows()]
                )

# Show the plot
fig.show()

#### State-wise trends.
Question: Is there a contrast in terms of states which announce Capex projects? 
Ans: Yes, based on 2014-22 data, Maharashtra, Rajasthan, Telangana, Uttarpradesh, Haryana, Karnataka, Andhra Pradesh, Tamilnadu, Gujarat and Odisha  are the top.

In [19]:

## XXXXXXX This code snippet will give Aggregate project (cost) for each state of India in Descending Order XXXXXX
# grouped_df = df[df['Fiscal_Year']=='FY19'].groupby(['State'])['Cost (Rs.million)'].sum().reset_index()
grouped_df = df.groupby(['State'])['Cost (Rs.million)'].sum().reset_index()

sorted_indices = sorted(range(len(grouped_df['Cost (Rs.million)'])), key=lambda k: grouped_df['Cost (Rs.million)'][k], reverse=True)
sorted_states = [grouped_df['State'][i] for i in sorted_indices]
sorted_values = [grouped_df['Cost (Rs.million)'][i] for i in sorted_indices]

# Create the bar chart
fig = go.Figure(go.Bar(x=sorted_states, y=sorted_values,))
fig.show()

In [27]:

print(df.columns)
# Perform pivot operation
pivot_df = df[df.Year<2020].pivot_table(index='State', columns='Project Status', values = 'Cost (Rs.million)', aggfunc='count')
print(pivot_df)
### Write Merged file to Excel 
out_folder_path = '/Users/kalyan/Library/CloudStorage/OneDrive-Personal/Kalyan/KK-Python/Kalyan-Jupyter-Notebooks/Capex/proc_data/'
out_file_name = 'Capex_Proj_Status2.xlsx'
out_file_path = os.path.join(out_folder_path, out_file_name)

pivot_df.to_excel((out_file_path), index = True)


Index(['Company Name', 'Project Name', 'Cost (Rs.million)', 'Project Status',
       'Last Updated On', 'Industry Group', 'Ownership Group', 'Project Type',
       'Company Code', 'Project No.', 'FY', 'Quarter', 'Type', 'Location',
       'District', 'State', 'Products', 'Capacity', 'Units',
       'Reason for Stalling of Projects', 'Sector', 'Owner Affliation',
       'Foreign', 'st_ut_code', 'region', 'IndG_L0', 'IndG_L4', 'IndG_L8',
       'Gross Fiscal Deficit', 'Revenue Deficit', 'Primary Deficit',
       'Own Tax Revenue', 'Own Non-Tax Revenue', ' Interest Payments',
       'State-wise Pension', 'State-wise Capital Expenditure',
       'Capital Outlay', 'Social Sector Expenditure',
       ' Gross State Domestic Product', 'GSVA-Agriculture',
       'GSVA-Manufacturing', 'GSVA-Construction', 'GSVA-Industry',
       'GSVA-Banking and Insurance ', 'GSVA-Services ',
       'Per Capita Net State Domestic Product', 'Net State Domestic Product',
       'NSVA-Agriculture', 'NSVA-Manufactu